In [11]:
import json

In [14]:
f = open('data.in', 'r')
dict = {}
lenTemp = []
for line in f:
    x = line.split(",")
    lenTemp.append(len(x))
        
set(lenTemp)
    
    
    
#     print(line, end='')


{1, 4, 5}